<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/01_cuda_lab/01_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple C Programming with Google Colab

- Instructir: Jeong-Gun Lee (School of Software, Hallym University)
- Email: jeonggun.lee@gmail.com
- Web: http://www.onchp.net


You can make a simple C code with the name of **codename.c** through [ %%writefile codename.c ] 

In [1]:
%%writefile prog.c

#include<stdio.h>

int main(void)
{
    printf("Hello World!~~~");
    
    return 0;  // 정상 완료
}

Writing prog.c


In [2]:
!ls

prog.c	sample_data


In [3]:
!gcc --version

gcc (Ubuntu 7.4.0-1ubuntu1~18.04) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [0]:
!gcc -o hello prog.c

In [5]:
!./hello

Hello World!~~~

*  *  *
## Pthread Exercise
*  *  *



In [6]:
%%writefile pthread.c

/////////////////////////////////////////////////////////
// POSIX pthread example
// adapted from
// https://computing.llnl.gov/tutorials/pthreads/
//
// gcc thread_1.c -o th -pthread
/////////////////////////////////////////////////////////

 #include <pthread.h>
 #include <stdio.h>
 
 #define NUM_THREADS 5

 /////////////////////////////////////////////////////////
 //  print function used as all threads
 /////////////////////////////////////////////////////////
 void *PrintHello(void *threadid)
 {
    int tid;
    tid = (int)threadid;
    printf("Hello World! It's me, thread #%d!\n", tid);
	
    pthread_exit(NULL);
 }

 /////////////////////////////////////////////////////////
 // start the threads and print in MAIN
 /////////////////////////////////////////////////////////
 int main (int argc, char *argv[])
 {
    // array of thread handles
	pthread_t threads[NUM_THREADS];
	// create return code
    int rc;
	// thread counter
    int t;

	// launch the threads and note that the print order 
	// between MAIN and the threads is
	// not deterministic and may change when you SSH in versus
	// use serial console, and from run to run
    for(t=0; t<NUM_THREADS; t++){
       printf("In main: creating thread %d\n", t);
       rc = pthread_create(&threads[t], NULL, PrintHello, (void *)t);
       if (rc){
          printf("ERROR; return code from pthread_create() is %d\n", rc);
       } // if (rc)	
    } //for(t=0; t<NUM_THREADS; t++)
    /* Last thing that main() should do */
    pthread_exit(NULL);
 } // main

Writing pthread.c


In [7]:
!gcc -o pthread pthread.c -lpthread

pthread.c: In function ‘PrintHello’:
pthread.c:21:11: warning: cast from pointer to integer of different size [-Wpointer-to-int-cast]
     tid = (int)threadid;
           ^
pthread.c: In function ‘main’:
pthread.c:45:59: warning: cast to pointer from integer of different size [-Wint-to-pointer-cast]
        rc = pthread_create(&threads[t], NULL, PrintHello, (void *)t);
                                                           ^


In [8]:
time !./pthread

In main: creating thread 0
In main: creating thread 1
In main: creating thread 2
Hello World! It's me, thread #0!
Hello World! It's me, thread #1!
In main: creating thread 3
In main: creating thread 4
Hello World! It's me, thread #2!
Hello World! It's me, thread #3!
Hello World! It's me, thread #4!
CPU times: user 37.2 ms, sys: 8.7 ms, total: 45.9 ms
Wall time: 1.63 s


In [9]:
%%writefile matrixmul.c

#include <pthread.h>
#include <stdlib.h>
#include <stdio.h>
 
#define SIZE 1024   // Size by SIZE matrices
int num_thrd;   // number of threads
 
int A[SIZE][SIZE], B[SIZE][SIZE], C[SIZE][SIZE];
 
// initialize a matrix
void init_matrix(int m[SIZE][SIZE])
{
  int i, j, val = 0;
  for (i = 0; i < SIZE; i++)
    for (j = 0; j < SIZE; j++)
      m[i][j] = val++;
}
 
void print_matrix(int m[SIZE][SIZE])
{
  int i, j;
  for (i = 0; i < SIZE; i++) {
    printf("\n\t| ");
    for (j = 0; j < SIZE; j++)
      printf("%2d ", m[i][j]);
    printf("|");
  }
}
 
// thread function: taking "slice" as its argument
void* multiply(void* slice)
{
  int s = (int)slice;   // retrive the slice info
  int from = (s * SIZE)/num_thrd; // note that this 'slicing' works fine
  int to = ((s+1) * SIZE)/num_thrd; // even if SIZE is not divisible by num_thrd
  int i,j,k;
 
  printf("computing slice %d (from row %d to %d)\n", s, from, to-1);
  for (i = from; i < to; i++)
  {  
    for (j = 0; j < SIZE; j++)
    {
      C[i][j] = 0;
      for ( k = 0; k < SIZE; k++)
        C[i][j] += A[i][k]*B[k][j];
    }
  }
  printf("finished slice %d\n", s);
  return 0;
}
 
int main(int argc, char* argv[])
{
  pthread_t* thread;  // pointer to a group of threads
  int i;
 
  if (argc!=2)
  {
    printf("Usage: %s number_of_threads\n",argv[0]);
    exit(-1);
  }
 
  num_thrd = atoi(argv[1]);
  init_matrix(A);
  init_matrix(B);
  thread = (pthread_t*) malloc(num_thrd*sizeof(pthread_t));
 
  // this for loop not entered if threadd number is specified as 1
  for (i = 1; i < num_thrd; i++)
  {
    // creates each thread working on its own slice of i
    if (pthread_create (&thread[i], NULL, multiply, (void*)i) != 0 )
    {
      perror("Can't create thread");
      free(thread);
      exit(-1);
    }
  }
 
  // main thread works on slice 0
  // so everybody is busy
  // main thread does everything if threadd number is specified as 1
  multiply(0);
 
  // main thead waiting for other thread to complete
  for (i = 1; i < num_thrd; i++)
    pthread_join (thread[i], NULL);

/*    
  printf("\n\n");
  print_matrix(A);
  printf("\n\n\t       * \n");
  print_matrix(B);
  printf("\n\n\t       = \n");
  print_matrix(C);
  printf("\n\n");
*/ 
  free(thread);
 
  return 0;
 
}

Writing matrixmul.c


In [10]:
!gcc -o matrixmul matrixmul.c -lpthread

matrixmul.c: In function ‘multiply’:
matrixmul.c:34:11: warning: cast from pointer to integer of different size [-Wpointer-to-int-cast]
   int s = (int)slice;   // retrive the slice info
           ^
matrixmul.c: In function ‘main’:
matrixmul.c:73:53: warning: cast to pointer from integer of different size [-Wint-to-pointer-cast]
     if (pthread_create (&thread[i], NULL, multiply, (void*)i) != 0 )
                                                     ^


In [11]:
time !./matrixmul 1

computing slice 0 (from row 0 to 1023)
finished slice 0
CPU times: user 120 ms, sys: 18.1 ms, total: 139 ms
Wall time: 16 s


In [12]:
time !./matrixmul 2

computing slice 0 (from row 0 to 511)
computing slice 1 (from row 512 to 1023)
finished slice 1
finished slice 0
CPU times: user 116 ms, sys: 21 ms, total: 137 ms
Wall time: 17.8 s


In [13]:
time !./matrixmul 4

computing slice 1 (from row 256 to 511)
computing slice 0 (from row 0 to 255)
computing slice 3 (from row 768 to 1023)
computing slice 2 (from row 512 to 767)
finished slice 3
finished slice 1
finished slice 2
finished slice 0
CPU times: user 118 ms, sys: 18.6 ms, total: 137 ms
Wall time: 16.5 s


In [14]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G